# The R workflow of Anne
---------------------------------------

In this jupyter-notebook, I'll show you the R workflow of Anne as the sample.

Hi Kei,
I don t know if that is the kind of workflow you were looking for, but here
is something I have been doing repeatedly lately:

- start a new session
- upload a network table, an edge attribute table and a node attribute table
(all in xls)
- select edges based on some edge attributes
- hide these edges
- select nodes based on some node attributes
- hide these nodes
- apply a layout the the network that is not hidden
- apply a visual style to this network; at this point I do every bit of the
visual style manually, but it is always the same. That is because I do not
know how to transfer a visual style from one session to the next yet. it
would be great if that was automated (I do it manually for one network and
then it is automatic for the following ones that have the same exact
attributes)

Best
Anne

# Setup
---------------------------------------
To execute this, please sutisfy following items.

- Java SE 8
- Cytoscape version 3.3+
- CyREST
- Use docker file to do this. The docker file provide you the environment below.
    - RCy3
    - igraph
    - ...

# Start a new session
---------------------------------------
To start a new session, you should do the following step.

### Steps
- Please execute cytoscape
- Import libraries
- Load Data
- Shaping data
- ...

In [2]:
library(RCy3)
library(igraph)

In [3]:
# Load Data
gal.table <- read.table('sampleData/galFiltered.sif',stringsAsFactors=FALSE)
head(gal.table)

V1,V2,V3
YKR026C,pp,YGL122C
YGR218W,pp,YGL097W
YGL097W,pp,YOR204W
YLR249W,pp,YPR080W
YLR249W,pp,YBR118W
YLR293C,pp,YGL097W


In [4]:
# create graph class
g <- new ('graphNEL', edgemode='directed')

# Get NodesVec
gal.table.nodevec <- unique(c(gal.table[[1]], gal.table[[3]]))

# add nodes to graph
for(node in gal.table.nodevec){
    g <- graph::addNode(node, g)
}

# get EdgeList
gal.table.fromvec = gal.table[[1]]
gal.table.tovec = gal.table[[3]]

for (index in 1:length(gal.table.fromvec)){
    g <- graph::addEdge (gal.table.fromvec[[index]] ,gal.table.tovec[[index]], g)
}

# show it in cytescape
cw <- CytoscapeWindow('vignette', , graph=g, overwrite=TRUE)
displayGraph (cw)
layoutNetwork (cw, layout.name='degree-circle')


Warning message:
In .local(from, to, graph): edges replaced: ‘YPL248C|YML051W’

[1] "label"


Then, you can get this image in cytoscape.

![cytoscape image](graph_image.jpeg)

# Select edges based on some edge attributes
---------------------------------------

### How to select edges based on their attributes?

First, we have to set attributes to edges. Then, we can select edges based on attributes.

### Method : selectEdges

#### Usage
selectEdges(obj, edge.names, preserve.current.selection=TRUE)

#### Arguments

- **obj** : a CytoscapeWindowClass object.  
- **edge.names** : a list of strings, the names of edges to select.  
- **preserve.current.selection** : a logical object.  




In [5]:
# get attributes data
gal.edge.attrs <- read.table('sampleData/galFiltered_edge.attrs',stringsAsFactors=FALSE, skip=1)
head(gal.edge.attrs)

V1,V2,V3,V4,V5
YKR026C,(pp),YGL122C,=,1
YGR218W,(pp),YGL097W,=,1
YGL097W,(pp),YOR204W,=,1
YLR249W,(pp),YPR080W,=,1
YLR249W,(pp),YBR118W,=,1
YLR293C,(pp),YGL097W,=,1


In [6]:
# initEdgeAttribute
g <- cw@graph
g <- initEdgeAttribute (graph=g, attribute.name='edgeType', attribute.type='char', default.value='unspecified')

# get EdgeAttrs
gal.edge.fromvec = gal.edge.attrs[[1]]
gal.edge.tovec = gal.edge.attrs[[3]]
gal.edge.attrs = gal.edge.attrs[[5]]

for (index in 1:length(gal.edge.fromvec)){
    edgeData(g, gal.edge.fromvec[[index]] ,gal.edge.tovec[[index]], 'edgeType') <- as.character(gal.edge.attrs[[index]])
}

cw@graph <- g
displayGraph (cw)

[1] "label"
[1] "edgeType"


In [7]:
# clear selection
clearSelection (cw)

In [8]:
# selection Edges
vec = c()
for (index in 1:length(gal.edge.fromvec)){
    if(gal.edge.attrs[[index]] == '1'){
        vec = c(vec, paste(gal.edge.fromvec[[index]], '(1)', gal.edge.tovec[[index]]))
    }
}
selectEdges (cw, vec)

Then, you can get the output,like the follwoing image, in cytoscape.

![cytoscape image](select_edge_image.jpeg)

# Hide these edges
---------------------------------------

If you want to hide selected edges, you can use this method.

### Method : hideSelectedEdges

#### Usage

hideSelectedEdges(obj)

#### Arguments

obj a CytoscapeWindowClass object.

In [9]:
# The code is only this!!
# TODO : this methods have some error. I can't do anything with this method.
hideSelectedEdges (cw)

NULL

# Select nodes based on some node attributes
---------------------------------------

### method : selectNodes

#### Usage
selectNodes(obj, node.names, preserve.current.selection=TRUE)

#### Arguments

- **obj** : a CytoscapeWindowClass object.
- **node.names** : a list of strings, the names of nodes to select.
- **preserve.current.selection** : a logical object.


In [10]:
# get attributes data
gal.node.attrs <- read.table('sampleData/galFiltered_node.attrs',stringsAsFactors=FALSE, skip=1)
head(gal.node.attrs)

V1,V2,V3
YKR026C,=,1
YGL122C,=,1
YMR146C,=,1
YDR429C,=,1
YFL017C,=,1
YOL123W,=,1


In [11]:
g <- cw@graph
g <- initNodeAttribute (graph=g, attribute.name='moleculeType', attribute.type='char', default.value='undefined')

# get EdgeAttrs
gal.node = gal.node.attrs[[1]]
gal.attrs = gal.node.attrs[[3]]

#for (index in 1:length(gal.node)){
#    nodeData (g, gal.node, 'moleculeType') <- as.character(gal.attrs[[index]])
#}

#cw <- setGraph (cw, g)
#displayGraph (cw)

In [12]:
# clear selection
clearSelection (cw)

In [13]:
vac = c()
for (index in 1:length(gal.node)){
    if(gal.attrs[[index]] == '1'){
        vec = c(vec, gal.node[[index]])
    }
}

selectNodes(cw, vec)

Then, you can get the output,like the follwoing image, in cytoscape.

![cytoscape image](select_nodes_image.jpeg)

# Hide these nodes
---------------------------------------
### Method : hideSelectedNodes

#### Usage

hideSelectedNodes(obj)

#### Arguments

obj : a CytoscapeWindowClass object.


In [14]:
# TODO : There are some error in this method. I can't do anything by this.
hideSelectedNodes(cw)



not yet implemented


# Apply a layout the the network that is not hidden
---------------------------------------
### Method : getLayoutNames, layoutNetwork

We use two method in this section. 'getLayoutNames' let us know available layoutNames and we can apply one of them by 'layoutNetwork' method.

#### 1. Usage
getLayoutNames(obj)

#### 1. Arguments
obj : a CytoscapeConnectionClass object.

#### 2. Usage
layoutNetwork(obj, layout.name= grid )

#### 2. Arguments
obj a CytoscapeWindowClass object.
layout.name a string, one of the values returned by getLayoutNames, ’grid’ by default.


In [131]:
# get available layout names
getLayoutNames(cw)

[1] "attribute-circle"      "stacked-node-layout"   "degree-circle"        
 [4] "circular"              "attributes-layout"     "kamada-kawai"         
 [7] "force-directed"        "grid"                  "hierarchical"         
[10] "fruchterman-rheingold" "isom"                  "force-directed-cl"

In [132]:
# execute layout that you want
layoutNetwork(cw, layout.name= 'isom')

Then, you can get the output,like the follwoing image, in cytoscape.

![cytoscape image](isom_graph_image.jpeg)

In [134]:
# execute layout that you want
layoutNetwork(cw, layout.name= 'kamada-kawai')

Then, you can get the output,like the follwoing image, in cytoscape.

![cytoscape image](kamada_kawai_image.jpeg)

# Apply a visual style to this network
---------------------------------------

At this point I do every bit of the visual style manually, but it is always the same.

That is because I do not know how to transfer a visual style from one session to the next yet.

It would be great if that was automated (I do it manually for one network and then it is automatic for the following ones that have the same exact attributes)'

In [135]:
#TODO


# Upload a network table, an edge attribute table and a node attribute table (all in xls)

---------------------------------------




In [ ]:
#TODO
